# Prohledávání hyperparametrů pro model BiLSTM nad datasetem TREC (coarse) 

Tento notebook slouží k nalezení optimálních hyperparametrů nad datasetem TREC (coarse) pro model BiLSTM s odemčenou embedding vrstvou. Hyperparametry jsou hledány pro původní i augmentovaný dataset pro normální trénink i destilaci.

K prohledávání je využito knihovny Optuna s algoritmem Hyperband. Nejlepší konfigurace je volena na základě F1-skóre, zkoušeno je 150 kombinací hyperparametrů pro každou z variant.

## Import knihoven a základní nastavení

In [1]:
from transformers import BasicTokenizer, Trainer
from datasets import concatenate_datasets, load_from_disk
import kagglehub
import optuna
import torch
import math
import base

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


Resetování náhodného seedu pro replikovatelnost výsledků.

In [2]:
base.reset_seed()

Ověření dostupnosti GPU.

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Načtení embeddingů.

Načtení datasetu a jeho základní předzpracování (tokenizace, vytvoření slovníků všech tokenů, vytvoření indexu pro GloVe embeddingy).

In [4]:
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [5]:
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "trec"

In [6]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits_coarse")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits_coarse")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits_coarse")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented_coarse")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits_coarse", f"~/data/{DATASET}/test-logits_coarse", f"~/data/{DATASET}/train-logits-augmented_coarse"]])
tokenizer = BasicTokenizer(do_lower_case=True)

Tokenizace.

In [7]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_train_data))

all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

Získání všech unikátních tokenů v datasetu.

In [8]:
vocab = base.get_vocab(all_data_tokens)

Přiřazení indexu jednotlivým tokenům.

In [9]:
word_index = dict(zip(vocab, range(len(vocab))))

Získání indexů z GloVe embeddingů.

In [10]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


Definice velikosti slovníku a velikosti embedding dimenze. 

In [11]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

8766


Vytvoření vazby mezi tokeny (jejich indexy) a embeddingy. Část tokenů nebyla nalezena, což ovšem nepředstavuje problém.

In [12]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 8551 words (215) misses


Přiřazení indexu tokenům v každé části datasetu.

In [13]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

all_train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),all_train_data_tokens))

Zarovnání délky všech záznamů.

In [14]:
train_padded_data = list(map(lambda x: base.padd(x,60), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,60), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,60), test_data_index))

all_train_padded_data = list(map(lambda x: base.padd(x,60), all_train_data_index))

Přidání ID tokenů do každé části datasetu.

In [15]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

all_train_data = all_train_data.add_column("input_ids", all_train_padded_data)

Základní konfigurace tréninku během prohledávání. Optuna nepracuje s epochami, ale s kroky. Níže je prováděn přepočet. 

Minimální délka tréninku je pět epoch, maximální 15 epoch. Maximální počet kroků pro warm up je nastaven na 10 % první epochy.

In [16]:
num_epochs = 15
batch_size = 128

In [17]:
data_length = len(train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

## Prohledávání s normálním tréninkem nad původním datasetem
Definice hledaných hyperparametrů a jejich rozmezí.

In [18]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [19]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Získání modelu s definovanou odemčenou embedding vrstvou. 

In [20]:
def get_BiLSTM():
    return base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=6, freeze_embed=False)

In [21]:
base.reset_seed()

Konfigurace jednotlivých tréninků.

In [22]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-embedd_coarse_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base-embedd_coarse_hp-search", epochs=num_epochs, batch_size=batch_size)

Konfigurace trenéra pro jednotlivé tréninky. 

In [23]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM()
)
  

Nastavení prohledávání.

In [24]:
best_trial_normal = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-embedd",
    n_trials=150
)

[I 2025-03-21 23:19:49,612] A new study created in memory with name: Base-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 3}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.587800,1.342690,0.479377,0.383846,0.370550,0.333946
2,1.134800,0.940124,0.661778,0.595080,0.554501,0.565719
3,0.773400,0.735816,0.733272,0.620977,0.626803,0.618899
4,0.579800,0.658441,0.768103,0.667579,0.653776,0.655763
5,0.495000,0.595582,0.794684,0.668816,0.679259,0.673455
6,0.422400,0.579615,0.798350,0.666671,0.683705,0.674230
7,0.357000,0.572807,0.814849,0.688591,0.693982,0.690160
8,0.318200,0.566859,0.804766,0.676982,0.688753,0.681398
9,0.275900,0.573188,0.812099,0.684657,0.692726,0.686888
10,0.241600,0.622248,0.792851,0.776659,0.696067,0.701843


[I 2025-03-21 23:20:46,213] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 0}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.663600,1.555050,0.372136,0.347339,0.274089,0.221895
2,1.401000,1.240159,0.538955,0.533411,0.431206,0.417814
3,1.094900,1.010397,0.632447,0.538683,0.542177,0.527703
4,0.879200,0.847275,0.703941,0.595488,0.600024,0.595060
5,0.742700,0.753364,0.742438,0.635664,0.632024,0.630945
6,0.648800,0.697164,0.755270,0.643076,0.642462,0.641522
7,0.573500,0.676837,0.769019,0.657320,0.654000,0.653377
8,0.520900,0.645463,0.779102,0.657941,0.666931,0.661627
9,0.487100,0.635496,0.784601,0.662483,0.670332,0.664511
10,0.445300,0.630282,0.776352,0.649701,0.664977,0.656785


[I 2025-03-22 02:03:31,488] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0020702773186767874, 'weight_decay': 0.003, 'warmup_steps': 2}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.099600,0.695343,0.765353,0.679235,0.646223,0.654243
2,0.477900,0.497923,0.828598,0.701320,0.705477,0.702598
3,0.252600,0.488373,0.850596,0.835506,0.787299,0.802977
4,0.116500,0.649239,0.835014,0.832742,0.785917,0.801405
5,0.056400,0.609378,0.861595,0.835926,0.823697,0.828508


[I 2025-03-22 02:03:54,486] Trial 149 pruned. 


In [25]:
print(best_trial_normal)

BestRun(run_id='66', objective=0.861541151855867, hyperparameters={'learning_rate': 0.004814585044790494, 'weight_decay': 0.001, 'warmup_steps': 4}, run_summary=None)


In [26]:
base.reset_seed()

## Prohledávání s destilací nad původním datasetem
Konfigurace jednotlivých tréninků.

In [27]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd_coarse_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd_coarse_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.


In [28]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [29]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [30]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

Nastavení prohledávání.

In [31]:
best_trial_distill = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill-embedd",
    n_trials=150
)

[I 2025-03-22 02:03:54,858] A new study created in memory with name: Distill-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 3, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.698800,3.221653,0.460128,0.391250,0.350303,0.302444
2,2.806000,2.332087,0.620532,0.571521,0.514177,0.524033
3,2.012400,1.802679,0.734189,0.622515,0.623736,0.619607
4,1.510600,1.587178,0.753437,0.651337,0.641145,0.640953
5,1.291800,1.417230,0.776352,0.655127,0.665547,0.658840


[I 2025-03-22 02:04:26,233] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 4, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.394500,1.438869,0.772686,0.669119,0.659525,0.656114
2,0.841800,0.924666,0.856095,0.719158,0.729966,0.722761
3,0.337700,0.800978,0.866178,0.880695,0.811218,0.834303
4,0.183700,0.719594,0.879927,0.893377,0.837260,0.859177
5,0.103100,0.678871,0.888176,0.898002,0.837383,0.859218
6,0.081500,0.700013,0.885426,0.895936,0.844959,0.864180
7,0.068700,0.670557,0.887259,0.898245,0.845592,0.866094
8,0.062600,0.659686,0.890009,0.900127,0.848823,0.868686
9,0.057100,0.661288,0.891842,0.900959,0.849592,0.869424
10,0.054600,0.665070,0.886343,0.895829,0.845337,0.864736


[I 2025-03-22 04:57:53,938] Trial 148 finished with value: 0.8647806323314217 and parameters: {'learning_rate': 0.004727837910315894, 'weight_decay': 0.002, 'warmup_steps': 0, 'lambda_param': 0.8, 'temperature': 3.5}. Best is trial 86 with value: 0.8693899540096316.


Trial 149 with params: {'learning_rate': 0.0039626784431113825, 'weight_decay': 0.001, 'warmup_steps': 0, 'lambda_param': 1.0, 'temperature': 3.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.245900,1.391083,0.766269,0.653136,0.657317,0.647293
2,0.864100,0.928021,0.849679,0.718921,0.721688,0.718588
3,0.365700,0.758522,0.870761,0.851910,0.785367,0.803589
4,0.188100,0.796020,0.877177,0.890310,0.826615,0.850078
5,0.113200,0.690984,0.879010,0.886758,0.820769,0.842896
6,0.087200,0.677488,0.881760,0.888292,0.823197,0.844945
7,0.072900,0.690107,0.869844,0.880963,0.813163,0.836099
8,0.062800,0.675188,0.879927,0.887606,0.821116,0.843480
9,0.057500,0.663805,0.882676,0.888987,0.823824,0.845478
10,0.053200,0.672285,0.880843,0.885765,0.823124,0.843545


[I 2025-03-22 04:58:47,951] Trial 149 pruned. 


In [32]:
print(best_trial_distill)

BestRun(run_id='86', objective=0.8693899540096316, hyperparameters={'learning_rate': 0.0047995795305536375, 'weight_decay': 0.004, 'warmup_steps': 1, 'lambda_param': 0.6000000000000001, 'temperature': 4.0}, run_summary=None)


Přepočet kroků s ohledem na změnu velikosti datasetu.

In [33]:
data_length = len(all_train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [34]:
base.reset_seed()

## Prohledávání s normálním tréninkem nad augmentovaným datasetem
Konfigurace jednotlivých tréninků.

In [35]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-embedd-aug_coarse_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base-embedd-aug_coarse_hp-search", epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí.

In [36]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [37]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace trenéra pro jednotlivé tréninky. 

In [38]:
trainer = Trainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

Nastavení prohledávání.

In [39]:
best_trial_normal_aug = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-aug-embedd",
    n_trials=150
)

[I 2025-03-22 04:58:48,246] A new study created in memory with name: Base-aug-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 23}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.758900,0.558416,0.813932,0.684080,0.694827,0.688067
2,0.232800,0.500471,0.849679,0.860809,0.797333,0.818076
3,0.115500,0.568789,0.856095,0.844311,0.813137,0.823016
4,0.066500,0.671564,0.849679,0.851058,0.815779,0.829720
5,0.043000,0.674286,0.857012,0.854608,0.812949,0.828012


[I 2025-03-22 04:59:40,025] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 4}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.423200,0.552985,0.850596,0.828859,0.810382,0.814391
2,0.065600,0.564440,0.860678,0.835434,0.836028,0.833924
3,0.028700,0.745886,0.844180,0.822149,0.813028,0.813689
4,0.016700,0.799053,0.857012,0.840254,0.820763,0.828346
5,0.012200,0.791594,0.861595,0.852755,0.825124,0.836063


[I 2025-03-22 11:06:19,213] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.002264170022337403, 'weight_decay': 0.008, 'warmup_steps': 0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.330900,0.599462,0.860678,0.841352,0.807662,0.819356
2,0.042100,0.655613,0.858845,0.824775,0.826364,0.823390
3,0.017400,0.780731,0.861595,0.836541,0.826604,0.829760
4,0.009700,0.862734,0.866178,0.831878,0.831101,0.830550
5,0.010300,0.956444,0.861595,0.858610,0.818912,0.832072
6,0.005200,0.969520,0.863428,0.853245,0.816688,0.831213
7,0.002600,1.166740,0.856095,0.814313,0.812725,0.812054
8,0.001300,1.215224,0.862511,0.841682,0.818234,0.826662
9,0.000800,1.296668,0.861595,0.840899,0.817912,0.825787
10,0.000600,1.261931,0.866178,0.819576,0.821320,0.819600


[I 2025-03-22 11:08:47,076] Trial 149 pruned. 


In [40]:
print(best_trial_normal_aug)

BestRun(run_id='115', objective=0.8510243730102385, hyperparameters={'learning_rate': 0.0021788274151528764, 'weight_decay': 0.007, 'warmup_steps': 5}, run_summary=None)


In [41]:
base.reset_seed()

## Prohledávání s destilací nad augmentovaným datasetem
Konfigurace jednotlivých tréninků.

In [42]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd-aug_coarse_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd-aug_coarse_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [43]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [44]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [45]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

Nastavení prohledávání.

In [46]:
best_trial_distill_aug = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill-aug-embedd",
    n_trials=150
)

[I 2025-03-22 11:08:47,381] A new study created in memory with name: Distill-aug-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 23, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.697400,1.142992,0.809349,0.676968,0.693155,0.683362
2,0.534400,0.857628,0.852429,0.715264,0.726921,0.719829
3,0.312400,0.759493,0.874427,0.882113,0.828034,0.845399
4,0.207100,0.705778,0.882676,0.890862,0.842184,0.860378
5,0.158400,0.688659,0.880843,0.891687,0.840080,0.859889
6,0.132900,0.673177,0.886343,0.895722,0.844874,0.864420
7,0.115800,0.666344,0.884510,0.891531,0.844876,0.861877
8,0.103100,0.661213,0.888176,0.893987,0.847952,0.865027
9,0.095700,0.646031,0.892759,0.898740,0.850890,0.868800
10,0.088400,0.637670,0.889093,0.896580,0.847936,0.866425


[I 2025-03-22 11:11:02,251] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 27, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.150700,0.891442,0.856095,0.718583,0.732011,0.722710
2,0.253100,0.649203,0.886343,0.897488,0.845307,0.865281
3,0.146600,0.657862,0.882676,0.891190,0.843599,0.860588
4,0.113900,0.632898,0.890009,0.896295,0.849118,0.866886
5,0.096100,0.625441,0.890926,0.896675,0.850170,0.867478
6,0.085000,0.583917,0.889093,0.895929,0.848192,0.866210
7,0.077300,0.594493,0.893676,0.900246,0.852418,0.870397
8,0.071500,0.595268,0.890926,0.896900,0.850529,0.867562
9,0.068200,0.590795,0.892759,0.899224,0.851821,0.869646
10,0.063900,0.583767,0.899175,0.904703,0.856391,0.874780


[I 2025-03-22 17:53:39,316] Trial 148 finished with value: 0.8723543968632019 and parameters: {'learning_rate': 0.0006899326854992335, 'weight_decay': 0.006, 'warmup_steps': 10, 'lambda_param': 0.7000000000000001, 'temperature': 5.5}. Best is trial 133 with value: 0.8881671330460716.


Trial 149 with params: {'learning_rate': 0.0013388240690482276, 'weight_decay': 0.007, 'warmup_steps': 7, 'lambda_param': 1.0, 'temperature': 5.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.875100,0.752546,0.870761,0.848678,0.816750,0.826369
2,0.168400,0.615927,0.897342,0.902300,0.855729,0.872544
3,0.110500,0.658747,0.879010,0.863572,0.841693,0.849248
4,0.089900,0.610170,0.888176,0.883692,0.848775,0.861841
5,0.078800,0.589700,0.893676,0.890790,0.861210,0.873618
6,0.072700,0.571215,0.901008,0.894695,0.858551,0.873052
7,0.066400,0.561674,0.898258,0.893865,0.864922,0.877096
8,0.061500,0.558819,0.900092,0.893816,0.857124,0.871670
9,0.057200,0.567727,0.901008,0.897566,0.867284,0.880123
10,0.054200,0.548837,0.900092,0.896639,0.866485,0.879287


Using the latest cached version of the module from /home/jovyan/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--f1/34c46321f42186df33a6260966e34a368f14868d9cc2ba47d142112e2800d233 (last modified on Fri Jan 10 23:14:01 2025) since it couldn't be found locally at evaluate-metric--f1, or remotely on the Hugging Face Hub.
[I 2025-03-22 17:58:10,111] Trial 149 finished with value: 0.8805127551471325 and parameters: {'learning_rate': 0.0013388240690482276, 'weight_decay': 0.007, 'warmup_steps': 7, 'lambda_param': 1.0, 'temperature': 5.0}. Best is trial 133 with value: 0.8881671330460716.


In [47]:
print(best_trial_distill_aug)

BestRun(run_id='133', objective=0.8881671330460716, hyperparameters={'learning_rate': 0.0011072530594019546, 'weight_decay': 0.008, 'warmup_steps': 12, 'lambda_param': 0.9, 'temperature': 5.5}, run_summary=None)
